<a href="https://colab.research.google.com/github/esleylab/machinelearning/blob/From-directory/learningtransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
%matplotlib inline

import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import tensorflow_datasets as tfds

# Baixando o dataset Cats vs Dogs
dataset_name = "cats_vs_dogs"
data, info = tfds.load(dataset_name, as_supervised=True, with_info=True)

# Dividindo em treino e teste
train_data = data['train'].take(int(info.splits['train'].num_examples * 0.8)) # Seleciona 80% dos dados de treino
test_data = data['train'].skip(int(info.splits['train'].num_examples * 0.2)) # Skip 80% para selecionar os 20% restantes para teste
processed = False

# Informações do dataset
#print(info)


In [53]:
import os
from PIL import Image
import tensorflow as tf
import numpy as np

import shutil

# Caminho da pasta que você deseja deletar
pasta_para_deletar = "/content/imagens/cats_dogs"
# Deletar a pasta e todo o seu conteúdo
try:
    shutil.rmtree(pasta_para_deletar)
    print(f"A pasta '{pasta_para_deletar}' foi deletada com sucesso.")
except FileNotFoundError:
    print(f"A pasta '{pasta_para_deletar}' não foi encontrada.")
except PermissionError:
    print(f"Sem permissão para deletar a pasta '{pasta_para_deletar}'.")
except Exception as e:
    print(f"Ocorreu um erro ao tentar deletar a pasta: {e}")

# Caminho da pasta onde as imagens serão salvas
base_dir = "./imagens/cats_dogs"
cats_dir = os.path.join(base_dir, 'cats')
dogs_dir = os.path.join(base_dir, 'dogs')
img_size=(128, 128)

# Criar os diretórios, se não existirem
os.makedirs(base_dir, exist_ok=True)
os.makedirs(cats_dir, exist_ok=True)
os.makedirs(dogs_dir, exist_ok=True)

def salvar_imagens(dataset, pasta_base):
    for i, (imagem, label) in enumerate(dataset.take(200)):  # Itera por todo o dataset
        # Redimensionar imagem para garantir que tenha tamanho adequado para salvar
        imagem = tf.image.resize(imagem, img_size)

        # Convert the image to uint8 before saving
        imagem = tf.cast(imagem * 255, dtype=tf.uint8) # Convert to uint8
        # Converter tensor para formato de imagem
        imagem_pil = Image.fromarray(imagem.numpy())  # Convertendo tensor para imagem PIL

        # Determinar a subpasta com base no rótulo
        subpasta = "cats" if label.numpy() == 0 else "dogs"
        caminho_pasta = os.path.join(pasta_base, subpasta)  # Criar a pasta 'cats' ou 'dogs'

        # Definir o nome e caminho do arquivo
        nome_arquivo = f"{subpasta}_{i}.jpg"
        caminho_arquivo = os.path.join(caminho_pasta, nome_arquivo)

        # Salvar a imagem
        imagem_pil.save(caminho_arquivo, "JPEG")

# Salvar imagens dos conjuntos de treino e teste
salvar_imagens(data['train'], base_dir)


A pasta '/content/imagens/cats_dogs' foi deletada com sucesso.


In [11]:
import shutil

# Caminho da pasta que você deseja deletar
pasta_para_deletar = "/content/imagens/cats_dogs"
# Deletar a pasta e todo o seu conteúdo
try:
    shutil.rmtree(pasta_para_deletar)
    print(f"A pasta '{pasta_para_deletar}' foi deletada com sucesso.")
except FileNotFoundError:
    print(f"A pasta '{pasta_para_deletar}' não foi encontrada.")
except PermissionError:
    print(f"Sem permissão para deletar a pasta '{pasta_para_deletar}'.")
except Exception as e:
    print(f"Ocorreu um erro ao tentar deletar a pasta: {e}")


A pasta '/content/imagens/cats_dogs' foi deletada com sucesso.


In [58]:
import os
import numpy as np
from PIL import Image

# Supondo que categories seja uma lista de subpastas com as categorias
# base_dir é o diretório onde as imagens estão armazenadas

categories = ['cats', 'dogs']  # Exemplo de categorias

data = []


# Função para carregar e processar imagens
def get_image(img_path):
    try:
        img = Image.open(img_path)
        img = img.convert('RGB')  # Garantir que a imagem tenha 3 canais (RGB)
        img = np.array(img)  # Converter a imagem para um array numpy
        return img, img.size  # Retornar imagem e tamanho
    except Exception as e:
        print(f"Erro ao carregar a imagem {img_path}: {e}")
        return None, None

# Iterar sobre todas as categorias e imagens
for c, category in enumerate(categories):
    category_path = os.path.join(base_dir, category)
    images = [os.path.join(category_path, f) for f in os.listdir(category_path)
              if os.path.splitext(f)[1].lower() in ['.jpg', '.png', '.jpeg']]

    print(f"Caminho: {category_path}, Quantidade de imagens: {len(images)}")

    for img_path in images:
        img, x = get_image(img_path)
        if img is not None:
            data.append({'x': img, 'y': c})  # Adiciona a imagem e o rótulo à lista cat = 0 dog = 1

# Contar o número de classes
num_classes = len(categories)
print(f"Número de classes: {num_classes}")


Caminho: ./imagens/cats_dogs/cats, Quantidade de imagens: 92
Caminho: ./imagens/cats_dogs/dogs, Quantidade de imagens: 108
Número de classes: 2


In [45]:
for item in data:
    print(item['x'])
    print(item['y'])

print(len(data))
print(categories)

0
['cats', 'dogs']
